In [ ]:
!rm -rf /kaggle/working/*

In [ ]:
#WITH 512X512 IMAGE 
import os
import torch
import shutil
from torch import nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from diffusers import UNet2DModel, DDPMScheduler, AutoencoderKL
from accelerate import Accelerator

# ======================
# Dataset Loader
# ======================
class MangoLeafDataset(Dataset):
    def __init__(self, folder):
        self.image_paths = [os.path.join(folder, f) for f in os.listdir(folder)
                            if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        return self.transform(img)

# ======================
# Paths
# ======================
class_name = "ANTHRACNOSE"
data_dir = f"/kaggle/input/gananthracnose/{class_name}"
output_dir = f"/kaggle/working/DiffusionModel_{class_name.replace(' ', '_')}"
os.makedirs(output_dir, exist_ok=True)

# ======================
# Load VAE
# ======================
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda").eval()

# ======================
# Model
# ======================
def get_unet():
    return UNet2DModel(
        sample_size=64,   # latent size for 512x512 VAE (512/8)
        in_channels=4,
        out_channels=4,
        layers_per_block=2,
        block_out_channels=(128, 256, 512, 512),
        down_block_types=("DownBlock2D", "DownBlock2D", "DownBlock2D", "AttnDownBlock2D"),
        up_block_types=("AttnUpBlock2D", "UpBlock2D", "UpBlock2D", "UpBlock2D")
    )

# ======================
# Dataset & DataLoader
# ======================
dataset = MangoLeafDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)

# ======================
# Training
# ======================
model = get_unet()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = DDPMScheduler(num_train_timesteps=1000)
accelerator = Accelerator()
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)

# Training loop
num_epochs = 100
model.train()

for epoch in range(1, num_epochs + 1):
    print(f"\U0001f9ea Epoch {epoch}/{num_epochs}")
    for batch in tqdm(dataloader, desc=f"Epoch {epoch}"):
        # Encode to latent space
        with torch.no_grad():
            latents = vae.encode(batch.to(accelerator.device)).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (latents.size(0),), device=latents.device).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        noise_pred = model(noisy_latents, timesteps).sample
        loss = nn.functional.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

    # Save checkpoint every 10 epochs
    if epoch % 10 == 0:
        ckpt_dir = os.path.join(output_dir, f"checkpoint_epoch_{epoch}")
        model.save_pretrained(ckpt_dir)
        print(f"\U0001f4be Saved checkpoint at: {ckpt_dir}")

    # Save sample image every 20 epochs
    if epoch % 20 == 0:
        model.eval()
        with torch.no_grad():
            sample_latents = torch.randn(1, 4, 64, 64).to(accelerator.device)
            for t in scheduler.timesteps:
                noise_pred = model(sample_latents, t).sample
                sample_latents = scheduler.step(noise_pred, t, sample_latents).prev_sample

            decoded_img = vae.decode(sample_latents / 0.18215).sample
            save_path = os.path.join(output_dir, f"sample_epoch_{epoch}.png")
            save_image(decoded_img, save_path, normalize=True)
            print(f"\U0001f5bc️ Saved sample image at: {save_path}")
        model.train()

# ======================
# Zip Final Model
# ======================
zip_path = shutil.make_archive(output_dir, 'zip', output_dir)
print(f"✅ Training complete. Zipped model at: {zip_path}")


2025-06-12 10:00:06.946747: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749722407.143638      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749722407.199314      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

🧪 Epoch 1/100


Epoch 1: 100%|██████████| 2500/2500 [26:47<00:00,  1.56it/s]


...

Epoch 7: 100%|██████████| 2500/2500 [24:08<00:00,  1.73it/s]


🧪 Epoch 8/100


Epoch 8: 100%|██████████| 2500/2500 [24:11<00:00,  1.72it/s]


🧪 Epoch 9/100


Epoch 9:  82%|████████▏ | 2041/2500 [19:44<04:26,  1.72it/s]

In [ ]:
#MULTI GPU CODE 256X256 LATENT DEFFUSION MODEL
import os
import torch
import shutil
from torch import nn
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from tqdm import tqdm
from diffusers import UNet2DModel, DDPMScheduler, AutoencoderKL
from accelerate import Accelerator

# ======================
# Dataset Loader
# ======================
class MangoLeafDataset(Dataset):
    def __init__(self, folder):
        self.image_paths = [os.path.join(folder, f) for f in os.listdir(folder)
                            if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.ToTensor(),
            transforms.Normalize([0.5]*3, [0.5]*3)
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img = Image.open(self.image_paths[idx]).convert("RGB")
        return self.transform(img)

# ======================
# Paths
# ======================
class_name = "ANTHRACNOSE256x256"
data_dir = f"/kaggle/input/gananthracnose256x256/{class_name}"
output_dir = f"/kaggle/working/DiffusionModel_{class_name.replace(' ', '_')}"
os.makedirs(output_dir, exist_ok=True)

# ======================
# Load VAE
# ======================
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda").eval()

# ======================
# Model
# ======================
def get_unet():
    return UNet2DModel(
        sample_size=32,   # 256 / 8 (latent size)
        in_channels=4,
        out_channels=4,
        layers_per_block=2,
        block_out_channels=(128, 256, 512, 512),
        down_block_types=("DownBlock2D", "DownBlock2D", "DownBlock2D", "AttnDownBlock2D"),
        up_block_types=("AttnUpBlock2D", "UpBlock2D", "UpBlock2D", "UpBlock2D")
    )

# ======================
# Dataloader
# ======================
dataset = MangoLeafDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)

# ======================
# Accelerate + Mixed Precision + Multi-GPU
# ======================
accelerator = Accelerator(mixed_precision="fp16")  # ⚡ Mixed precision
model = get_unet()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
scheduler = DDPMScheduler(num_train_timesteps=1000)
model, optimizer, dataloader = accelerator.prepare(model, optimizer, dataloader)

# ======================
# Training Loop
# ======================
model.train()
num_epochs = 100

for epoch in range(1, num_epochs + 1):
    print(f"\n🔁 Epoch {epoch}/{num_epochs}")
    for batch in tqdm(dataloader, desc=f"Epoch {epoch}", disable=not accelerator.is_local_main_process):
        with torch.no_grad():
            latents = vae.encode(batch.to(accelerator.device)).latent_dist.sample() * 0.18215

        noise = torch.randn_like(latents)
        timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (latents.size(0),), device=latents.device).long()
        noisy_latents = scheduler.add_noise(latents, noise, timesteps)

        noise_pred = model(noisy_latents, timesteps).sample
        loss = nn.functional.mse_loss(noise_pred, noise)

        optimizer.zero_grad()
        accelerator.backward(loss)
        optimizer.step()

    # Save checkpoint
    if epoch % 10 == 0 and accelerator.is_main_process:
        ckpt_dir = os.path.join(output_dir, f"checkpoint_epoch_{epoch}")
        model.save_pretrained(ckpt_dir)
        print(f"💾 Saved checkpoint: {ckpt_dir}")

    # Save sample image
    if epoch % 20 == 0 and accelerator.is_main_process:
        model.eval()
        with torch.no_grad():
            sample_latents = torch.randn(1, 4, 32, 32).to(accelerator.device)
            for t in scheduler.timesteps:
                noise_pred = model(sample_latents, t).sample
                sample_latents = scheduler.step(noise_pred, t, sample_latents).prev_sample

            decoded_img = vae.decode(sample_latents / 0.18215).sample
            save_path = os.path.join(output_dir, f"sample_epoch_{epoch}.png")
            save_image(decoded_img, save_path, normalize=True)
            print(f"🖼️ Saved sample image: {save_path}")
        model.train()

# ======================
# Final ZIP
# ======================
if accelerator.is_main_process:
    zip_path = shutil.make_archive(output_dir, 'zip', output_dir)
    print(f"✅ Training complete. Model zipped at: {zip_path}")

   
       
      

2025-06-12 14:03:30.240890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749737010.479461      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749737010.549967      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]


🔁 Epoch 1/100


Epoch 1: 100%|██████████| 1250/1250 [12:26<00:00,  1.68it/s]


...

Epoch 18: 100%|██████████| 1250/1250 [13:18<00:00,  1.57it/s]



🔁 Epoch 19/100


Epoch 19: 100%|██████████| 1250/1250 [13:29<00:00,  1.54it/s]



🔁 Epoch 20/100


Epoch 20:  93%|█████████▎| 1158/1250 [12:30<00:59,  1.54it/s]

In [ ]:
import os
import torch
import zipfile
from torchvision.utils import save_image
from diffusers import DDPMScheduler, AutoencoderKL, UNet2DModel
from tqdm import tqdm

# ==== Configuration ====
class_name = "GALL MILDGE DAMAGE"
model_dir = f"/kaggle/working/DiffusionModel_{class_name}"
output_dir = f"/kaggle/working/Generated_Latent_Images/{class_name}"
os.makedirs(output_dir, exist_ok=True)
num_images = 1000  # Adjust as needed

# ==== Load VAE ====
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse").to("cuda")
vae.eval()

# ==== Load trained UNet ====
unet = UNet2DModel.from_pretrained(model_dir).to("cuda")
unet.eval()

# ==== Load scheduler ====
scheduler = DDPMScheduler(num_train_timesteps=1000)

# ==== Generate Images ====
print(f"🎨 Generating {num_images} images for class: {class_name}")
for i in tqdm(range(num_images)):
    latents = torch.randn(1, 4, 64, 64).to("cuda")

    for t in scheduler.timesteps:
        with torch.no_grad():
            noise_pred = unet(latents, t).sample
        latents = scheduler.step(noise_pred, t, latents).prev_sample

    with torch.no_grad():
        decoded_image = vae.decode(latents).sample

    save_path = os.path.join(output_dir, f"{class_name}_{i:05}.png")
    save_image(decoded_image, save_path, normalize=True)

print(f"✅ Image generation complete. Images saved in: {output_dir}")

# ==== Zip the Generated Images ====
zip_path = f"/kaggle/working/{class_name}_Generated.zip"
print(f"📦 Zipping folder {output_dir} → {zip_path}")
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, output_dir)
            zipf.write(file_path, arcname=arcname)

print(f"✅ Zipped dataset saved at: {zip_path}")
